# Embedding Documents and storing them in vector store
We will use FAISS for storing vectors.

In [1]:
import os
from pathlib import Path

try:
    file_path = Path(__file__).resolve()
except NameError:
    # Fallback for notebooks
    file_path = Path.cwd()

os.chdir(file_path.parent)

In [2]:
from config.settings import MODEL, EMBEDDING_MODEL, DATA_DIR, VECTOR_DB_DIR

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader,UnstructuredMarkdownLoader, TextLoader

## Load Documents

In [3]:
loader = DirectoryLoader(
    path=DATA_DIR,
    glob="**/*.md",
    loader_cls=lambda filename: TextLoader(filename, encoding="utf-8")
)
docs = loader.load()

Add table_name in metadata

In [4]:
for doc in docs:
    doc.metadata['table_name'] = doc.metadata['source'].split('\\')[-1].split('.')[-2]

## Creating Vectors and storing in DB

In [5]:
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

In [6]:
vector_store = FAISS.from_documents(docs, embeddings)

## Store Vector DB locally

In [7]:
vector_store.save_local(VECTOR_DB_DIR)